# The Validity of the SoVi Index
### Seth E.  Spielman, David Folch, Joseph Tuccillo

This is a script to calculate the Social Vulnerability (Sovi) Index at multiple geographic scales and usign different sets of input variables.  The Sovi index assess a places social vulnerability to natural hazards and has been used in hundreds of publications, in both the academic and policy domains.  We show that this index fails to meet certain intuitive assumptions about the nature of measurement of social vulnerability.

The aim is to illustrate instability in the index and problems with *convergent validity*.  Convergent Validity is the idea that multiple measurements of the same thing, using a valid instruments, should yield similar *absolute* and *relative* measurements.  For example two thermometers measuing the same cup of water should yield the same approximate temperature- this would be an example of validitiy of absolute measurement.  An less rigorous concept of validity is to consider three cups ordered from hottest (_**A**_) to coldest (_**C**_), the two thermometers would be valid in a *relative* sense if their measurements of cups A, B, C differed absolutely (they measured different temperatures for each cup) but still placed cup _**A**_ as the warmest and cup _**C**_ as the coldest.

We will show in this analysis that the Sovi Index fails this "cup test" that is it often mixes up the orders of the cup.  Counties in the United States that are high vulnerability at one scale of measurement (or form the index) are often low vulnerability in a slightly different version of the index.

## Variables and Components
The Sovi Index is constructed using a tecnhnique called Principal Components Analysis, this is matrix decomposition method that uses the covariance matrix of the input data.  Usually, in the social sciences one treats the "compents" what come of out a PCA as latent variables.  For example, in Sovi it comon to fine components that measure things like "race and class".  In this analysis we also show that this latent variable approach has maked some underlying problems with the Soci index, namely that variables contribute to the index in ways that are profoundly counter intuitive.  

## There is a paper
For an in-depth discussion of these ideas please see the companion paper to this anlysis [URL]() or contact the suthors. 

## Data Prep

In this section we read in data from the American Community Survey and the Decennial Census and processes the varaibles such that they correspond to the inputs used to commonly construct the Sovi Index,  There is **a lot of data wrangling here**, combining variables into new variables, computing standard errors, etc.  Its all farily straightforward and I will not talk through the details here.

In [1]:
import os
import pandas as pd
import pysal as ps
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats.mstats import zscore as ZSCORE
from scipy.stats import rankdata
from scipy.stats import spearmanr 
from spss_pca_dl import SPSS_PCA

local_path = '/Users/dlee/data/repo/sovi-validity'
# os.chdir(local_path + '/code')
path = local_path + '/code'
outPath = local_path +'/data'

### Functions for calculating Standard Errors

THe functions below are to calculate the standard error (SE) of various types of estimates from the American Cummunity Survey.  These are provided by the US Census Bureau in the ACS Techncial Documentation.

In [2]:
#SE of a sum
def se_sum(*ses):
    df_temp = pd.DataFrame(list(ses))
    df_temp = df_temp.T
    df_temp = np.square(df_temp)
    df_temp = df_temp.sum(1)
    return np.sqrt(df_temp)

#SE of a ratio
def se_ratio(est, estd, sen, sed):
    sen2 = np.square(sen)
    sed2 = np.square(sed)
    est2 = np.square(est)
    num = np.sqrt(sen2 + (est2*sed2))
    return num / estd

#SE of a proprotion
def se_prop(est, estd, sen, sed):
    sen2 = np.square(sen)
    sed2 = np.square(sed)
    est2 = np.square(est)
    num = sen2 - (est2*sed2)
    num_alt = sen2 + (est2*sed2)
    problems = num <= 0
    num[problems] = num_alt[problems]
    num = np.sqrt(num)
    return num / estd

### Analysis

Here we load the data `db1` or alternative add a refernce to db1 with a new name `US_ALL`. We describe each of the input variables with a human readable name and outline their expected contributions to vulnerability.  For example, the variable `MEDAGE_ACS`, which measures the median age in the county is expected to have a positive ("pos") contribution to social vulnerability - the logic is that older populations are more vulnerable to disasters.   Similarly the variable `QRICH200K`, which measures the portion of hosuing units making over $200k/year is expected to have a negative ("neg") contribution to social vulnerability.   

In [3]:
#Import Data
# US_All = db1.copy()
# US_All['Geo_FIPS'] = US_All.index.values
US_All = pd.read_csv(local_path+'/data/input/sovi_inputs.csv')
US_All.index = US_All.Geo_FIPS

# attribute name and expected influence on vulnerability
input_names = [['MEDAGE_ACS','pos','person','Median Age'],
               ['BLACK_ACS','pos','person','Pop African-American (%)'],
               ['QNATAM_ACS','pos','person','Pop Native American (%)'],
               ['QASIAN_ACS','pos','person','Pop Asian (%)'],
               ['QHISP_ACS','pos','person','Pop Hispanic (%)'],
               ['QAGEDEP_ACS','pos','person','Age Dependency (%)'],
               ['QPUNIT_ACS','pos','person','Persons Per Housing Unit'],
               ['PRENTER_ACS','pos','hu','Rental Housing (%)'],
               ['QNRRES_ACS','pos','person','Nursing Home Residents (%)'],
               ['QFEMALE_ACS','pos','person','Pop Female (%)'],
               ['QFHH_ACS','pos','hu','Female-Headed Households (%)'],
               ['QUNOCCHU_ACS','pos','hu','Vacant Housing (%)'],
               ['PERCAP_ALT','neg','person','Per-Capita Income'],
               ['QESL_ALT','pos','person','English as Second Language (%)'],
               ['QCVLUN','pos','person','Unemployment (%)'],
               ['QPOVTY','pos','person','Poverty (%)'],
               ['QMOHO','pos','hu','Mobile Homes (%)'],
               ['QED12LES_ALT','pos','person','Adults Completed <Grade 12 (%)'],
               ['QFEMLBR','pos','person','Female Employment (%)'],
               ['QEXTRCT_ALT','pos','person','Extractive Sector Employment (%)'],
               ['QSERV_ALT','pos','person','Service Sector Employment (%)'],
               ['QSSBEN','pos','hu','Social Security Income (%)'],
               ['QNOAUTO_ALT','pos','hu','No Automobile (%)'],
               ['QFAM','neg','person','Children in Married Families (%)'],
               ['QRICH200K','neg','hu','Annual Income >$200K (%)'],
               ['MDGRENT_ALT','neg','hu','Median Rent'],
               ['MHSEVAL_ALT','neg','hu','Median Home Value'],
               ['POPDENS','pos','person','Population Density']] 

#Get attribute names
attr_names=[j[0] for j in input_names]
#cols = [c for c in US_All.columns if c.find('_SE') == -1]
attr_names.append('Geo_FIPS')
#US_All = US_All.dropna(axis=0) #two counties misisng data in state 15 and 48
US_All = US_All[attr_names]
US_All['stateID'] = US_All.Geo_FIPS.str.slice(0,3,1)
attr_names.remove('Geo_FIPS')

#### Flipping Signs

To ensure that each variable contributes as expected to the final Sovi Index following Eric Tate (2012?) we flip the signs of the input data.  Variables where a high value are expected to contribute negatively to Social vulnerability have their signs flipped, such that positive values become negative.  This has the effect of reversing the sign of the scores used to compute the index.  Consider the variable measuring the percent of households making over \$200,000 per year, if the mean at the county-level in a state was 5\% a county two SD above the mean, say one where 10\% of the population made mroe than $200K, that would have a positive z-score (+2).  However, this high prevalance of wealthy people should reduce the vulnerability, by flipping the signs we ensure that that mean is now -5\% and a value of -10\% is two standard deviations *below* the mean.  Thus when multiplied by its (positive) loading a county whit high wealth will have a lower social vulnerability index.

In [4]:
# input data prep
# --swap signs of the attributes expected to have a "negative" affect on vulnerability
for name, sign, sample, hrname in input_names:
    if sign == 'neg':
        US_All[name] = -US_All[name].values
    elif sign == 'pos':
        pass
    else:
        raise Exception("problem")

## Build FEMA Region, State, and National Files.  
## Create data frames to hold analysis results

In this block of code we do two things.  First, we create subsets of the national file for each FEMA region.  Second, we create pandas data frames to hold the results of the drop 1 and analysis and each of the 

In [5]:
########################################
##SOVI FOR FEMA REGIONS
#########################################
#Build FEMA subRegions Dict values= state ID's
FEMA_subs= {'FEMA_1':['g23','g50','g33','g25','g09','g44']}
FEMA_subs['FEMA_2'] = ['g36','g34']
FEMA_subs['FEMA_3'] = ['g42','g10','g11','g24','g51','g54']
FEMA_subs['FEMA_4'] = ['g21','g47','g37','g28','g01','g13','g45','g12']
FEMA_subs['FEMA_5'] = ['g27','g55','g26','g17','g18','g39']
FEMA_subs['FEMA_6'] = ['g35','g48','g40','g05','g22']
FEMA_subs['FEMA_7'] = ['g31','g19','g20','g29']
FEMA_subs['FEMA_8'] = ['g30','g38','g56','g46','g49','g08']
FEMA_subs['FEMA_9'] = ['g06','g32','g04']
FEMA_subs['FEMA_10'] = ['g53','g41','g16']

#Dict to hold variable loadings
varContrib = {}

#Multiindexed DataFrame to hold all FEMA SOVI Scores
#Level 1 of the index is the fema region.
#Level 2 of the FEMA Region is the county FIPS
geoLevels = US_All.Geo_FIPS
# femaLevels = FEMA_subs.keys()     # PYTHON 2
femaLevels = list(FEMA_subs)       # PYTHON 3
geoLabels = []
femaLabels = []
for f in femaLevels:
    femaRegionIndexes = US_All[US_All['stateID'].isin(FEMA_subs[f])].index.values
    geoLabels.extend([US_All.index.get_loc(i) for i in femaRegionIndexes])
    femaLabels.extend(np.repeat(femaLevels.index(f), len(femaRegionIndexes)))

US_femaSub_Multi_Index = pd.MultiIndex(levels=[femaLevels, geoLevels], 
                                    labels=[femaLabels, geoLabels], 
                                    names=['FEMA_Region', 'Geo_FIPS'])
#In the FEMA_Region_Sovi_Score data frame ranks are BY FEMA REGION.  
#The data frame holds both the SOVI score and the county rank
#This means that there should be 10 counties with rank 1 (one for each FEMA Region)
FEMA_Region_Sovi_Score = pd.DataFrame(index=US_femaSub_Multi_Index, columns=['sovi', 'rank']) 


#Create New England conglomerate of states
#These are the FIPS codes for the states wit hthe letter "g" appended
US_All.loc[US_All.stateID.isin(['g23','g33','g25']), 'stateID'] = 'g23g33g25'

#These are the states in the state level analysis
stateList = ['g23g33g25', 'g36','g51','g13','g17','g48','g29','g46','g06','g16']

#In the FEMA_State_Sovi_Score data frame ranks are BY STATE.  
#The data frame holds both the SOVI score and the county rank
#This means that there should be 10 coutnies with rank 1 (one for each state in stateList)
FEMA_State_Sovi_Score = pd.DataFrame(index=US_All.index[US_All.stateID.isin(stateList)], columns=['sovi', 'rank']) 

US_All[US_All.stateID.isin(stateList)]

,MEDAGE_ACS,BLACK_ACS,QNATAM_ACS,QASIAN_ACS,QHISP_ACS,QAGEDEP_ACS,QPUNIT_ACS,PRENTER_ACS,QNRRES_ACS,QFEMALE_ACS,...,QSERV_ALT,QSSBEN,QNOAUTO_ALT,QFAM,QRICH200K,MDGRENT_ALT,MHSEVAL_ALT,POPDENS,Geo_FIPS,stateID
Geo_FIPS,,,,,,,,,,,,,,,,,,,,,
g06001,36.6,0.119930,0.002850,0.262336,0.224033,0.177094,2.752589,0.428662,0.003825,0.509620,...,0.157549,0.222696,0.104629,-0.737959,-0.097571,-1265,-514900.0,2050.203696,g06001,g06
g06003,43.2,0.000000,0.192147,0.026734,0.065163,0.172932,3.072727,0.040771,0.009190,0.441103,...,0.228782,0.366234,0.046753,-0.791209,-0.036364,-755,-371300.0,1.621221,g06003,g06
g06005,48.4,0.025050,0.018722,0.012525,0.123742,0.243804,2.334926,0.191319,0.006567,0.456043,...,0.236278,0.424698,0.038564,-0.736941,-0.028870,-1039,-286500.0,63.513410,g06005,g06
g06007,37.3,0.013548,0.007778,0.041776,0.142008,0.210940,2.515680,0.353125,0.005116,0.504950,...,0.225212,0.344994,0.068623,-0.661976,-0.021417,-885,-237400.0,134.497917,g06007,g06
g06009,49.5,0.010284,0.007779,0.013602,0.104116,0.256092,2.411913,0.139663,0.002725,0.497418,...,0.188910,0.407079,0.036792,-0.758644,-0.039048,-1043,-276200.0,44.614181,g06009,g06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
g51800,38.2,0.415693,0.000914,0.015092,0.029282,0.185986,2.728641,0.233162,0.004025,0.519379,...,0.158816,0.272867,0.050620,-0.642960,-0.040489,-982,-248900.0,210.451768,g51800,g51
g51810,34.8,0.186227,0.002027,0.061964,0.066810,0.173459,2.610738,0.324347,0.002469,0.509735,...,0.166596,0.226608,0.038727,-0.703976,-0.045096,-1224,-272700.0,1765.057856,g51810,g51
g51820,38.2,0.108226,0.004327,0.009796,0.063956,0.236472,2.391230,0.363131,0.003566,0.521160,...,0.190891,0.346806,0.112205,-0.598729,-0.029425,-761,-166800.0,1398.365183,g51820,g51


## Calculate Sovi

In [6]:
i = list(FEMA_subs)[0]
#Subset FEMA subregion
FEMARegionData=US_All[US_All['stateID'].isin(FEMA_subs[i])]

# compute SoVI
inputData = FEMARegionData.drop(['Geo_FIPS','stateID'], axis = 1, inplace = False)
pca = SPSS_PCA(inputData, reduce=True, varimax=True)



NodeException: Got negative eigenvalues: [-9.88792381e-17  3.87971117e-16  3.22327441e-04  8.65785723e-04
  3.33987271e-03  9.52792536e-03  1.30000503e-02  1.62577399e-02
  2.11487820e-02  3.58742536e-02  4.53501254e-02  5.63941030e-02
  7.83863616e-02  8.51782687e-02  1.23323549e-01  1.33400263e-01
  1.96620545e-01  2.30962851e-01  2.87824934e-01  5.19357350e-01
  6.02939422e-01  7.95353625e-01  1.04435137e+00  1.11858611e+00
  2.03537363e+00  3.08136700e+00  6.54985047e+00  1.19919664e+01].
You may either set output_dim to be smaller, or set reduce=True and/or svd=True

In [11]:
for i in FEMA_subs:
    
    #Subset FEMA subregion
    FEMARegionData=US_All[US_All['stateID'].isin(FEMA_subs[i])]

    # compute SoVI
    inputData = FEMARegionData.drop(['Geo_FIPS','stateID'], axis = 1, inplace = False)
    pca = SPSS_PCA(inputData, reduce=True, varimax=True)
    sovi_actual = pca.scores_rot.sum(1)
    sovi_actual = pd.DataFrame(sovi_actual, index=FEMARegionData.Geo_FIPS, columns=['sovi'])
    attrib_contribution = pca.weights_rot.sum(1)
    
    FEMA_Region_Sovi_Score.loc[i, 'sovi'] = sovi_actual.values
    ranks_tmp = pd.DataFrame(abs(FEMA_Region_Sovi_Score.loc[i, 'sovi']).rank(method='average'))
    ranks_tmp.columns = ['rank']
    FEMA_Region_Sovi_Score.loc[i, 'rank'] = 1 + (len(FEMA_Region_Sovi_Score.loc[i,'rank']) - ranks_tmp.values) 

        
    ##Write attribute contribution output     
    #Generate dictionary for all net loadings by variable and region
    varContrib[i]=zip(attr_names,attrib_contribution.tolist())

NodeException: Got negative eigenvalues: [-9.88792381e-17  3.87971117e-16  3.22327441e-04  8.65785723e-04
  3.33987271e-03  9.52792536e-03  1.30000503e-02  1.62577399e-02
  2.11487820e-02  3.58742536e-02  4.53501254e-02  5.63941030e-02
  7.83863616e-02  8.51782687e-02  1.23323549e-01  1.33400263e-01
  1.96620545e-01  2.30962851e-01  2.87824934e-01  5.19357350e-01
  6.02939422e-01  7.95353625e-01  1.04435137e+00  1.11858611e+00
  2.03537363e+00  3.08136700e+00  6.54985047e+00  1.19919664e+01].
You may either set output_dim to be smaller, or set reduce=True and/or svd=True

## Compute National SoVI

In [29]:
# DONGHOON
#######################
##Compute National SoVI
#######################
# compute SoVI
inputData = US_All.drop(['Geo_FIPS','stateID'], axis = 1, inplace = False)
# inputData.to_hdf('./inputdata.hdf', 'data')
pca = SPSS_PCA(inputData, reduce=True, varimax=True)
sovi_actual_us = pca.scores_rot.sum(1)
sovi_actual_us = pd.DataFrame(sovi_actual_us, index=US_All.Geo_FIPS, columns=['sovi'])
attrib_contribution_us = pca.weights_rot.sum(1)

#redundancy w/ranks here to preserve compatibility
US_All_Full_Sovi_Rank = abs(sovi_actual_us).apply(rankdata, axis=0, method='average')
sovi_actual_us['rank'] = abs(sovi_actual_us).apply(rankdata, axis=0, method='average')
sovi_actual_us['rank'] = 1 + (len(sovi_actual_us['rank']) - sovi_actual_us['rank'])

#Generate dictionary for all net loadings by variable and region
varContrib['USA']=zip(attr_names,attrib_contribution_us.tolist())

In [31]:
attr_names, attrib_contribution_us

(['MEDAGE_ACS',
  'BLACK_ACS',
  'QNATAM_ACS',
  'QASIAN_ACS',
  'QHISP_ACS',
  'QAGEDEP_ACS',
  'QPUNIT_ACS',
  'PRENTER_ACS',
  'QNRRES_ACS',
  'QFEMALE_ACS',
  'QFHH_ACS',
  'QUNOCCHU_ACS',
  'PERCAP_ALT',
  'QESL_ALT',
  'QCVLUN',
  'QPOVTY',
  'QMOHO',
  'QED12LES_ALT',
  'QFEMLBR',
  'QEXTRCT_ALT',
  'QSERV_ALT',
  'QSSBEN',
  'QNOAUTO_ALT',
  'QFAM',
  'QRICH200K',
  'MDGRENT_ALT',
  'MHSEVAL_ALT',
  'POPDENS'],
 array([ 0.21133895, -0.26623081,  0.494398  , -0.08977324,  0.56264829,
         0.76040652,  0.01645677,  0.16755052,  0.33146194,  0.73694898,
         0.20776048,  0.27974765,  0.27911943,  0.48155516, -0.15012316,
         0.34339951, -0.36726431,  0.1400679 ,  0.50562425,  0.11459834,
         0.6476124 ,  0.40225286,  0.39932333,  0.29389029,  0.21529479,
         0.22647339,  0.14497283,  0.15507241]))

## Compute SoVI for selected state(s) in each FEMA Region

In [25]:
#############################################
##State Analysis     
#############################################
for st in stateList:
    #Subset FEMA subregion
    stateData=US_All[US_All.stateID == st]

    # compute SoVI
    inputData = stateData.drop(['Geo_FIPS','stateID'], axis = 1, inplace = False)
    pca = SPSS_PCA(inputData, reduce=True, varimax=True)
    sovi_actual = pca.scores_rot.sum(1)
    sovi_actual = pd.DataFrame(sovi_actual, index=stateData.Geo_FIPS, columns=['sovi'])
    attrib_contribution = pca.weights_rot.sum(1)
    
    
    #sovi_alt_computation = (zinputs * attrib_contribution).sum(1) # this is just a check
    #sovi_alt_computation = pd.DataFrame(sovi_alt_computation, columns=['sovi'])
    #if not np.allclose(sovi_actual, sovi_alt_computation):
    #   raise Exception, "mismatch"
    ######################
'''##Compute spearman correlation
######################
##DO THIS USING THE USA AND STATE?FEMA REGION SPECIFIC SOVI

#compute US base ranks for all drop 1 correlations
spearmanr(US_SoVI_Drop1_Score[])

#rankContrib = (28-rankContrib) + 1
#compare Fema regions and US

#compare fema regions and states

#compare states and US
US_SoVI_Drop1_Score.
#compare drop 1 to full sovi
scipy.stats.mstats.spearmanr'''

##Write attribute contribution output     
#Generate dictionary for all net loadings by variable and region
varContrib[st]=zip(attr_names,attrib_contribution.tolist())

In [28]:
#############################################
##Consolidate variable net contribs and ranks 
#############################################
netContribCols = varContrib.keys()

netContrib = pd.DataFrame(columns=netContribCols, index=attr_names)

for r in varContrib.keys():
    for name, value in varContrib[r]:
        netContrib.loc[name][r] = value
netContrib

netContrib = netContrib.reindex(netContrib.USA.abs().sort(inplace=False, ascending=False).index)

AttributeError: 'Series' object has no attribute 'sort'

In [57]:
#############################################
##Consolidate variable net contribs and ranks 
#############################################
netContribCols = varContrib.keys()

netContrib = pd.DataFrame(columns=netContribCols, index=attr_names)

for r in varContrib.keys():
    for name, value in varContrib[r]:
        netContrib.loc[name][r] = value

netContrib = netContrib.reindex(netContrib.USA.abs().sort(inplace=False, ascending=False).index)

#reorder table        
cols = ['USA', 'FEMA_1', 'g23g33g25', 
'FEMA_2', 'g36','FEMA_3', 'g51', 'FEMA_4', 'g13', 'FEMA_5', 'g17',
'FEMA_6', 'g48', 'FEMA_7', 'g29', 'FEMA_8', 'g46', 'FEMA_9', 'g06', 'FEMA_10', 
'g16']
netContrib = netContrib[cols]

#variable rank using absolute value      
rankContrib = abs(netContrib).apply(rankdata, axis=0, method='average')
rankContrib = (28-rankContrib) + 1

combContrib = pd.DataFrame(columns=list(netContrib.columns), index=list(netContrib.index))
#can't think of a more elegant way to do this
for aRow in range(netContrib.shape[1]):
    for aCol in range(netContrib.shape[0]):
        combContrib.ix[aCol][aRow] = str(round(netContrib.ix[aCol][aRow], 2)) + ' (' + str(int(rankContrib.ix[aCol][aRow])) + ')'

#build list of varIDs and human readable names
#sort and use as index for conContrib
nameSort = [[name, hrname] for name, sign, sample, hrname in input_names]
nameSort = pd.DataFrame(nameSort)
nameSort.index = nameSort.loc[:,0]
nameSort = nameSort.reindex(list(combContrib.index))    
    
#set descriptive names
combContrib.index = list(nameSort.loc[:,1])

#write out results
combContrib

AttributeError: 'Series' object has no attribute 'sort'

###Compute Correlation

In this section we compute spearman correlation for differe subsets of counties.  This is our check of the concurrent validity of of the metric. 

In [ ]:
######################
##Compute spearman correlation
######################
##DO THIS USING THE USA AND STATE?FEMA REGION SPECIFIC SOVI

#compute US base ranks for all drop 1 correlations
spearmanr(US_SoVI_Drop1_Score[])

#rankContrib = (28-rankContrib) + 1
#compare Fema regions and US

#compare fema regions and states

#compare states and US
US_SoVI_Drop1_Score.
#compare drop 1 to full sovi
scipy.stats.mstats.spearmanr

##Drop One Variable (DOV) analysis

Here we drop a single variable at a time from the data set and compute the Sovi score with n-1 variables.  This gives us 28 different sovi scores, each one representing an index that is slightly different from the one constructed in the previous sectiob index.  We say "slightly" different, because each of the 28 different sovi indices is missing a single variable.  However, removing only one of the 28 variables shouldn't have a dramatic impact on the index, especailly if the variable wasn't very important to begin with.  

The code below does some fancy indexing in Pandas.  Its an elegant way to store 28 national sovi indices in a single data frame.  

In [ ]:
#Construct table to hold the results of the drop one analysis
#Sort variable list based on importance rank.
USvarRanks = rankContrib.USA.copy() #have to make a copy to sort index
USvarRanks.sort('USA')
dropLevels = USvarRanks.index

#build multindex
geoLevels = US_All.Geo_FIPS
geoLabels = []
for _ in range(len(dropLevels)):
    geoLabels.extend(range(len(geoLevels)))
dropLabels = np.repeat(range(len(dropLevels)), len(geoLevels))

US_Drop1_Multi_Index = pd.MultiIndex(levels=[dropLevels, geoLevels], 
                                    labels=[dropLabels, geoLabels], 
                                    names=['DroppedVar', 'Geo_FIPS'])
                                    
US_Drop1_NetContrib = pd.DataFrame(index=dropLevels, columns=dropLevels)                     

US_SoVI_Drop1_Score = pd.DataFrame(index=US_Drop1_Multi_Index, columns=['sovi']) 


#Compute drop-one 
for j in dropLevels:
    US_dropj = US_All.drop([j,'Geo_FIPS', 'stateID'], axis = 1, inplace = False)
    pca = SPSS_PCA(US_dropj, reduce=True, varimax=True)
    sovi_actual = pca.scores_rot.sum(1)
    sovi_actual = pd.DataFrame(sovi_actual, index=geoLevels, columns=['sovi'])
    US_SoVI_Drop1_Score.loc[j, 'sovi'] = sovi_actual.values
    attrib_contribution = pd.DataFrame(data=pca.weights_rot.sum(1), index=US_dropj.columns)
    #print(j +" " + str(np.isnan(attrib_contribution.values).sum()))
    attrib_contribution = attrib_contribution.transpose()
    attrib_contribution.index = [j]
    #print(attrib_contribution.loc[j,:])
    US_Drop1_NetContrib.loc[j, attrib_contribution.columns] = attrib_contribution.loc[j,:] #.values

#Sort descriptive labels
USvarRanks = rankContrib.USA.copy()
USvarRanks.index = desc
USvarRanks.sort('USA')
US_Drop1_NetContrib.index = USvarRanks.index
US_Drop1_NetContrib.columns = USvarRanks.index
    
US_Drop1_NetContrib = US_Drop1_NetContrib.T #T so columns indexes dropped variable.




In [ ]:
US_Drop1_NetContrib=US_Drop1_NetContrib.convert_objects(convert_numeric=True)
US_Drop1_NetContrib = US_Drop1_NetContrib.apply(lambda x: np.round(x, 2))

In [ ]:
%matplotlib inline
sns.set_context("poster")

#Reorder and apply variable description to labels
#USvarRanks = rankContrib.USA.copy() #have to make a copy to sort index
#USvar
#dropLevels = USvarRanks.indexdesc

#plt.figure(figsize=(20, 16))
mask=np.isnan(US_Drop1_NetContrib)
sns.heatmap(US_Drop1_NetContrib, annot=True, linewidths=.25, vmin=-1, vmax=1, annot_kws={"size": 7})
